# Contents : Reveiw Translate
## 1.라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import datetime as dt
import os

import warnings
warnings.filterwarnings('ignore')

## 2. 폰트불러오기

In [2]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(Colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

In [3]:
pd.options.display.max_columns=30
pd.options.display.max_rows=30

## 3. 데이터불러오기

In [4]:
glob.glob('*.*')

['1.RFM_Analysis.ipynb',
 '2.Review_Translate.ipynb',
 '2.Review_WordCloud.ipynb',
 'Review01_translations.csv',
 'Review02_translations.csv',
 'Review03_translations.csv',
 'Review04_translations.csv',
 'Review05_translations.csv']

In [5]:
# 원본 데이터들
path = glob.glob('../PJT_01/NanoDegree_PJT1/1.EDA/data/*.*')
path

['../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_closed_deals_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_customers_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_geolocation_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_marketing_qualified_leads_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_orders_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_order_items_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_order_payments_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_order_reviews_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_products_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_sellers_dataset.csv',
 '../PJT_01/NanoDegree_PJT1/1.EDA/data\\product_category_name_translation.csv']

In [6]:
df_orders = pd.read_csv('../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_orders_dataset.csv')
df_order_reviews = pd.read_csv('../PJT_01/NanoDegree_PJT1/1.EDA/data\\olist_order_reviews_dataset.csv')

## 4. 데이터셋 병합

In [7]:
df_reviews = pd.merge(df_orders, df_order_reviews, left_on='order_id', right_on='order_id')
df_reviews.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51


In [8]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   order_id                       100000 non-null  object
 1   customer_id                    100000 non-null  object
 2   order_status                   100000 non-null  object
 3   order_purchase_timestamp       100000 non-null  object
 4   order_approved_at              99839 non-null   object
 5   order_delivered_carrier_date   98207 non-null   object
 6   order_delivered_customer_date  97013 non-null   object
 7   order_estimated_delivery_date  100000 non-null  object
 8   review_id                      100000 non-null  object
 9   review_score                   100000 non-null  int64 
 10  review_comment_title           11715 non-null   object
 11  review_comment_message         41753 non-null   object
 12  review_creation_date           100000 non-nul

### 형변환

In [9]:
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews["review_answer_timestamp"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       100000 non-null  object        
 1   customer_id                    100000 non-null  object        
 2   order_status                   100000 non-null  object        
 3   order_purchase_timestamp       100000 non-null  object        
 4   order_approved_at              99839 non-null   object        
 5   order_delivered_carrier_date   98207 non-null   object        
 6   order_delivered_customer_date  97013 non-null   object        
 7   order_estimated_delivery_date  100000 non-null  object        
 8   review_id                      100000 non-null  object        
 9   review_score                   100000 non-null  int64         
 10  review_comment_title           11715 non-null   object        
 11  r

In [10]:
df_reviews.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-11
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18,2018-08-18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03,2017-12-03
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17,2018-02-17


## 5. 리뷰데이터 번역 준비

In [11]:
# 설치 pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [12]:
# translator.translate(번역할 단어)
# 결과값 src="번역전언어", dest = "번역후언어", text= 번역된 언어 (주로 영어로 세팅되어 있음)
# # translator.translate(번역할 단어).text하면 결과만 나옴
translator = Translator()

In [13]:
# test
translator.translate("Muito bom o produto.").text
#dest가 번역후 언어인데, 기본으로 english가 세팅되어있음

'The product is very good.'

In [14]:
# test
translator.translate("Muito bom o produto.", dest = "ko").text


'제품은 매우 좋습니다.'

## 6. 리뷰스코어별, 리뷰 가져오기 

In [15]:
df_reviews.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [16]:
s_m = df_reviews[['review_score', 'review_comment_message']]

In [17]:
s_m.isnull().sum()

review_score                  0
review_comment_message    58247
dtype: int64

In [18]:
sm = s_m.dropna()
sm

,review_score,review_comment_message
0,4,"Não testei o produto ainda, mas ele veio corre..."
1,4,Muito bom o produto.
3,5,O produto foi exatamente o que eu esperava e e...
6,2,fiquei triste por n ter me atendido.
10,1,Aguardando retorno da loja
...,...,...
99990,5,Otimo comprar por este site as mercadorias che...
99991,1,"Ele não é um mini cajon, é um shaker, ou seja,..."
99996,4,So uma peça que veio rachado mas tudo bem rs
99997,5,Foi entregue antes do prazo.


In [19]:
# ??for i in range(len(sm['review_comment_message'])):
#     sm['review_comment_message']
    

In [20]:
Rscore01 = sm[sm['review_score']==1]
Rscore01.head(2)

,review_score,review_comment_message
10,1,Aguardando retorno da loja
26,1,Até o momento não recebi o produto Protetor De...


In [21]:
Rscore01['review_score'].value_counts()

1    9179
Name: review_score, dtype: int64

In [22]:
Rscore02 = sm[sm['review_score']==2]
Rscore03 = sm[sm['review_score']==3]
Rscore04 = sm[sm['review_score']==4]
Rscore05 = sm[sm['review_score']==5]

In [23]:
Rscore05.head(100)

,review_score,review_comment_message
3,5,O produto foi exatamente o que eu esperava e e...
15,5,Obrigado pela atenção. Lojas lannister perfeit...
24,5,Sempre vou comprar aqui pois é a melhor parabéns
27,5,"Lindos. \r\nAcabamento incrivel, muiro macios ..."
33,5,Ficou tudo ok só pensei q fosse maior mas.ta bom
...,...,...
551,5,Produto recebido com sucesso!
553,5,entrega rapida
554,5,Entrega feita em menos de uma semana;\r\nProdu...
556,5,chegou antesdo programado


In [24]:
Rscore01.shape, Rscore02.shape, Rscore03.shape, Rscore04.shape, Rscore05.shape

((9179, 2), (2229, 2), (3665, 2), (6034, 2), (20646, 2))

## 7. 번역하기

In [25]:
translator.translate(Rscore01['review_comment_message'].to_list()[1]).text

"So far I have not received the protective product of crib mattress - waterproof cotton, I've made the complaint since 18/06 and has not yet gotten return.I will cancel the order and reversal the purchase hj"

In [26]:
Rscore01.head(5)

,review_score,review_comment_message
10,1,Aguardando retorno da loja
26,1,Até o momento não recebi o produto Protetor De...
31,1,Foi entregue um produto divergente do que eu h...
41,1,"Inicialmente, na data da compra o produto era ..."
45,1,A cuba veio quebrada. Como efetuo barroca?


In [27]:
Rscore01['review_comment_message'].to_list()[4]

'A cuba veio quebrada. Como efetuo barroca?'

- 시간이 너무오래걸려서 Sampling이 필요함

In [28]:
translations = []
for i in range(len(Rscore01['review_comment_message'])):
    try: 
        translations.append(translator.translate(Rscore01['review_comment_message'].to_list()[i]).text)
    except:
        translations.append('E_R_R_O_R')
translations

In [29]:
# frac
R01S = Rscore01.sample(frac = 0.3, random_state = 42)
R01S

,review_score,review_comment_message
7018,1,"O produto veio diferente da q mostra na foto, ..."
88700,1,O produto não funciona e veio errado.
99880,1,"Meu pedido não foi entregue, porém já recebi e..."
77826,1,nunca tive problemas emrelaçao a entrega so qu...
20430,1,"Comprei o produto no dia 12/07, previsão de en..."
...,...,...
70084,1,Primeira vez que faço um pedido pela Internet ...
88218,1,"Não recebi meu produto, muito menos o dinheiro..."
18865,1,Eu comprei 3 unidades do pendente de só veio 1...
34829,1,Comprei um tapete de 2X2.5 e recebi um de 2X1....


In [31]:
translations1 = []
for i in range(len(R01S['review_comment_message'])):
    try: 
        translations1.append(translator.translate(R01S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations1.append('E_R_R_O_R')
translations1

['이 제품은 사진의 Q 쇼와 다르고, 사치스럽지 않으며, 그대로 패딩되지 않았습니다.',
 '제품이 작동하지 않고 잘못되었습니다.',
 '내 주문은 전달되지 않았지만 이미 전자 메일이 배달되었다는 이메일을 받았으며 주문을 원합니다.',
 '나는 이번에는 그들이 나에게주지 않았다는 사실 만 배달에 문제가 없었습니다.',
 '07/12에 제품을 구입하여 07/30에 배송 예측을 구입했습니다.지금까지 성적은 발행되지 않았습니다.나는 이미 첫 번째 할부를 지불했으며 제품을받지 못했습니다.아무도 내가받을 답을주지 않습니다.',
 '나는 당신에게 연락을 시도하고 있지만 사이트를 통해주는 것은 아닙니다.석탄이 나오고 촛불이 부러졌습니다.나는 그것의 교환을 요청합니다.제품은 마감일 전에도 도착했습니다.감사',
 '회사는 내 제품을 제공하지 않았습니다.그는 20 일 동안 판매 할 수있는 부품이 없다고 경고하기 위해 기다렸다.카드 구매를 기다리는 일주일 이상이 있습니다.',
 '배송을위한 30 일 지연은 언제 및 요청 된 주문 정보입니다. 나는 마감일을 요구하는 Lannister 매장으로부터 자동 이메일 만 받았으며 문제를 해결하기 위해 더 많은 마감일을 요청했습니다.최악의 경험이었습니다',
 '고객을 속이는 것은 사진을 찍는 다른 제품을 얻는다. 내가 돈을 가져 갔고 공정하게 갔다면 나쁜 자료는 자료가 훨씬 나아질 것입니다. 불행히도 그것이 일어나는 일입니다.',
 '제품의 분리 및 배송 과정에서 품질 부족.그들은 lannister.com 고객 제품을 내 주소로 보냈고 내 주문은 Lannister의 고객 주소로 보냈습니다.',
 '나는 두 장을 사서 설명이나 의사 소통없이 단 하나의 상자를 받았다.더 나쁜 스타크를 만드는 것은 이러한 사례를 해결하기 위해 통신 채널 P를 제공하지 않습니다.단지 평가 또는 이메일.:(',
 '나는 05 컵을 사서 01 잔만 받았다. 내가 구매 한 다른 안경의 배달을 요청하고 지불했다.',
 '주문을 추적 할 때 이미 수신자에게 배달 된 것으로

In [32]:
R02S = Rscore02.sample(frac = 0.3, random_state = 42)
R03S = Rscore03.sample(frac = 0.3, random_state = 42)
R04S = Rscore04.sample(frac = 0.3, random_state = 42)
R05S = Rscore05.sample(frac = 0.3, random_state = 42)

In [33]:
translations2 = []
for i in range(len(R02S['review_comment_message'])):
    try: 
        translations2.append(translator.translate(R02S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations2.append('E_R_R_O_R')
translations2

['나는 팔없이 팔을 샀다.이유.테이블 아래에 맞습니다.그들은 나에게 팔을 주었다.',
 '나는 우체국에서 주문을 찾아야했다.그들은 다른 배송 시스템이 그렇게하기 때문에 전화로 우리에게 연락하지 않았습니다.',
 '지체에서 나는 이미 매장을 추천하지 않는다. 이미 내가 사서 나에게 보냈지 만 만족하지 않았기 때문이다.',
 '나는 거의 한 달 전에 구매를했고 승자는 배달 시간 동안 일주일 미만의 제품을 게시했습니다.모든 것이 정상이고 대답하지 않는지 깨닫는 것 외에도.',
 '매우 나쁜 품질.적합하지 마십시오',
 'Felpuda는 얇은 층, 광고 잎이 필요하지 않으며 소비자 코드를 침해합니다.',
 '품질이 좋지 않은 종이에는 몇 개의 구겨진 잎이 함께 제공되었으며 잘 자랐습니다.',
 '제품이 이미 배달되었다고 기록되었지만 아무것도 도착하지 않았습니다. \\',
 '배달 된 제품은 구매 한 제품에 해당하지 않았습니다',
 '불만 1',
 '나는 같은 모델의 두개골을 가지고 있었고, 품질은 비교할 수없고 완벽한 마무리 세부 사항, 즐겁고 강력한 사운드 경험, 특히베이스의 하이라이트 .... 이것은 두개골이 아닙니다.',
 '나는 속도를 좋아했고, 제품은 내가 주문한 방식으로 왔지만 영화는 모두 깨졌습니다.제품의 가격으로 인해 우편물을 지불 할 가치가 없기 때문에 다시 보내지 않을 것입니다.',
 '나는 제품을 좋아했지만 그는 많은 물과 결혼하고 다른 사람들이 있었고 결혼하고있었습니다.',
 '전체 요청이 도착하지 않았으며, 요청에는 3 개가 있지만 2 개만 도착했습니다.',
 '나는 양호한 것을 발견했다.',
 '제품은 크게 말하지만 아이에게 맞지 않습니다!',
 '금속 조각과 녹이있는 나사.',
 '제품이 도착했을 때 기다렸습니다.',
 '예정된 마감일 이전에 잘 도착했지만 드론 H18 항목은 호환되지 않았습니다.내 드론의 입구는 JST입니다.구매 한 제품을 구매하기 위해 구매할 어댑터가 있습니까?',
 '주문의 일부만이 배송되었고 제품은 같은 상점에서 나

In [34]:
translations3 = []
for i in range(len(R03S['review_comment_message'])):
    try: 
        translations3.append(translator.translate(R03S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations3.append('E_R_R_O_R')
translations3

['마감일 이전에 도착했을 때, 재료는 마감 문제에서 품질이 좋지 않지만 더 이상 좋지 않습니다.',
 '이 사이트에서 상품은 함께 모여졌지만 이것은 오지 않았습니다.',
 '모두에게 OBG',
 '제 생각에는 우체국의 분배 지점에서 상품을 제거해야하기 때문에 배송비를 청구 할 필요가 없습니다.',
 '제품을받지 못했고 수익 전에 요청시 취소되었습니다.',
 '아주 좋은, 마감일, 배송 및 제품',
 '더 잘 평가할 수 있기를 기다리고 있습니다',
 '조각은 요청 된대로 왔지만 조치는 요청 된대로 나오지 않았습니다.',
 '나는 이것이 브라질의 주 쓰레기 인 우체국의 잘못이지만 제품에 대한 유일한 불만입니다.',
 '나는 내가 원하는대로 제품을 올바르게 받았다. 나는 접착제의 결함이있는 부분과 함께 제공된 유리 필름에 실망했다.',
 '쓰레기는 면봉과 매우 작은 더러운면을위한 것입니다.',
 '나는 모든 것이 크다고 생각했다. 모든 것이 잘되었다',
 '제품은 매우 나쁘다.\r\n\r\n매우 파주 토 제품',
 '그것은 머리카락에 약간의 기름진 모습을 남긴다. 그러나 평평한 철이 조금 더해지고 부드럽게 만들고 맛있는 냄새가 나지만 제품 자체는 이상하다 ...',
 '사이트에서 본 것보다 더 낮은 제품을 발견했지만 잘 제공됩니다.',
 '나는 추천한다 !!그러나 내 제품은 아직 배달되지 않았습니다 ... 1 개월 이상이 있습니다.해결 해주세요 !!!참고 : 7 명이 당신과 함께 샀고 나만 도착하지 않았습니다 !!!!!!',
 '그것은 다른 기능들 사이에서 시간 조정과 관련하여 내 기대를 많이 만났습니다.',
 '모든 것이 잘됩니다',
 '나는 2 개의 호스를 샀고 단지 1만이왔다',
 '나는 아직 제품을받지 못했고 연락 할 수 없었습니다.',
 '확인',
 '원래 향수, 아주 좋습니다.그러나 15 일 늦게 배달 지연이 많이있었습니다.',
 '전달은 최고 품질로 발생했지만.초고속.그러나 제품 자체는 실망했습니다.불평하지는 않지만 이미지는 다른 유형의 제품을 제안했

In [35]:
translations4 = []
for i in range(len(R04S['review_comment_message'])):
    try: 
        translations4.append(translator.translate(R04S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations4.append('E_R_R_O_R')
translations4

['좋은 제품.',
 '제 시간에 도착했습니다.좋은 제품.',
 '재료는 더 저항력이 있어야합니다. 나는 작은 깨지기 쉬운 것을 발견했습니다 ...',
 '엄청난.',
 '나는 Lannister Stores에서 샀지 만 자유 시장 패키지에 들어 와서 매우 불편하게 만들었습니다.',
 '아직 사용하지는 않았지만 추천합니다',
 '마감일 전에 제품을 받았습니다.Lannister Ball Show.',
 '지갑 덮개가 정확했지만 유리 필름은 장치보다 작았습니다.',
 '항상 정시에 그리고 영원한 품질로',
 '양질의 제품이지만 브라질에서는 매우 비쌉니다.여행하는 데 시간이 걸리는 방법은 여기서 바로 사기로 결정했습니다 ...',
 '엄청난',
 '제품이 적고 빠르게 제공됩니다',
 '매우 좋은',
 '예상대로 제공되는 제품.매우 좋은!',
 '마감일 전에 도착했습니다',
 '나는 제품에 매우 만족했고 예측 날짜 이전에 배달 된 신속하게 배달되었습니다.',
 '매우 좋은',
 '내 딸은 내가 추천하는 아름다운 도시락을 좋아했습니다',
 '모든 것이 괜찮습니다',
 '지지에 결함이 없으면 우수 할 수 있습니다.그것은 단지 지원이었습니다.나머지는 완벽합니다',
 '빠른',
 '제품은 설명과 이미지에 따라 왔지만 2 개월이 걸렸습니다.',
 '빠르고 단순하며 가볍습니다',
 '좋은 제품, 영업 세일즈맨은 전문적입니다',
 '제품은 빠르고 완벽한 상태로 도착했습니다.감사!',
 '아름다운 키트.빠른 배달.추천합니다',
 '성공적으로 제공되는 제품은 이제 보증 기간 내에 품질을 측정하는 데 사용됩니다.',
 '광고에서와 같이 제품, 좋은 세일즈맨',
 '아직 제품을 시도하지 않았습니다.',
 '추천합니다',
 '제품 중 하나는 지연이 있었지만 장군은 제대로 왔습니다.',
 '제품은 사이트에서 좋은 가격으로 발견되었으며 매우 빠르게 전달되었습니다.',
 '나는 흰색 시계를 사서 빨간색을 보냈다 ...',
 '놓친 제품',
 '모두 순서대로 신뢰할 수 있습니다.',
 '나는 서비

In [36]:
translations5 = []
for i in range(len(R05S['review_comment_message'])):
    try: 
        translations5.append(translator.translate(R05S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations5.append('E_R_R_O_R')
translations5

['모든 시간에',
 '나는 이미 Lannister Stores에서 많은 제품을 구입했습니다.그러나 나는 왜 당신의 제품으로부터의 제안으로 e-EMA를받지 않는지 모르겠습니다.\r\n당신은 진심으로.',
 '비 호환성으로 인해 돌아올 것입니다. 훌륭한 제품!',
 '내가 사전 -그들이하는 일에 많은 진지한 사람들이 있습니다 ...',
 '나는 마감일 전에 그것을 받았다 !!!잘 포장되었습니다!',
 '확인',
 '매니큐어의 멋진 건조기!슈퍼 추천',
 '훌륭한',
 '훌륭한 품질의 제품!',
 '빠르고 실용적입니다.',
 '나는 가방, 훌륭한 품질과 아름다운 것을 좋아했습니다.',
 '매우 좋은',
 '빠른 배달',
 '마감일 이전에 제공되는 제품.',
 '아름다운 제품과 마감일 전에 내가 좋아했습니다.',
 '추천합니다',
 '나는 상품 배달 속도를 좋아했고, 잘 포장되어 마감일 이전에 그들은 축하합니다.',
 '추천합니다',
 '훌륭한 공급 업체, 훌륭한 인수.',
 '이 제품은 제 시간에 도착하여 완벽하게 도착했습니다.시계는 아름답다 !!나는 모두에게 가게를 추천합니다.',
 '모든 것이 순서대로',
 'BOM',
 '매우 효율적으로 매우 빠르게 전달됩니다',
 '예상보다 일찍 도착했습니다',
 '매우 좋은!마감일 전에 배달!',
 '가격을위한 좋은 제품\r\n그러나 잘 맞지 않는 탈착식 부품을 향상시킬 수 있습니다.\r\n나머지는 괜찮습니다',
 '나는 Baratheon에서 구매를 좋아한다 .... 너희들은 천',
 '엄청난',
 '아주 좋은 품질의 제품!',
 '완벽한 아름다운 제품, 적절한 시간에 왔습니다.',
 '훌륭한 제품, 신뢰할 수있는 딜러.추천합니다',
 '매우 만족스럽고 빠른 배송.',
 '내 딸은 그것을 좋아했고, 그녀는 그녀의 방에서 아름다웠다.',
 '제공되는 가격에 좋은 제품.',
 '합의 된 마감일 내에 배달됩니다',
 '훌륭한 품질의 제품!',
 '훌륭한',
 '훌륭한',
 '마감일 전에 제품이 잘 도착했습니다.',
 '구매할 

In [39]:
translations01 = pd.DataFrame(translations1)
translations02 = pd.DataFrame(translations2)
translations03 = pd.DataFrame(translations3)
translations04 = pd.DataFrame(translations4)
translations05 = pd.DataFrame(translations5)

In [40]:
# csv 파일로 저장합니다.
translations01.to_csv("Review01_translations.csv", index=False, encoding="utf-8")
translations02.to_csv("Review02_translations.csv", index=False, encoding="utf-8")
translations03.to_csv("Review03_translations.csv", index=False, encoding="utf-8")
translations04.to_csv("Review04_translations.csv", index=False, encoding="utf-8")
translations05.to_csv("Review05_translations.csv", index=False, encoding="utf-8")

In [41]:
pd.read_csv("Review05_translations.csv", encoding="utf-8")

,0
0,모든 시간에
1,나는 이미 Lannister Stores에서 많은 제품을 구입했습니다.그러나 나는 ...
2,비 호환성으로 인해 돌아올 것입니다. 훌륭한 제품!
3,내가 사전 -그들이하는 일에 많은 진지한 사람들이 있습니다 ...
4,나는 마감일 전에 그것을 받았다 !!!잘 포장되었습니다!
...,...
6189,좋은 거래
6190,매우 좋은 !제 시간에 도착했습니다!사랑했던 !
6191,예상 마감일 전에 받았습니다.
6192,나는 그것이 잘 작동하는 것을 좋아했습니다


In [ ]:
for i in range(len(sm['review_comment_message'])):
    sm['review_comment_message'][i] = sm['review_comment_message'].to_list()[i].replace('\r\n','')
sm